# Explorative Daten Analyse - V1

In [7]:
# imports 
import pandas as pd
import matplotlib.pyplot as pylt
import seaborn as sns

ModuleNotFoundError: No module named 'matplotlib'

In [4]:
test = pd.read_pickle("/Users/janavihs/projects/anomaly-detection-software/data/LIDS/CVE-2014-0160/interim/test.pkl")

In [6]:
test.netword_send.describe()

count       46.000000
mean      7084.608696
std       9915.598209
min          0.000000
25%          0.000000
50%       2906.000000
75%      13281.500000
max      33826.000000
Name: netword_send, dtype: float64